# Import Modules

In [1]:
import pandas as pd
import numpy as np
from ray import tune
import torch
from torch import nn
from torch import optim

import utils

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Data

In [3]:
vegetable = utils.VEGETABLES[0]

if vegetable == "なましいたけ":
    train_size = 3000
elif vegetable == "セルリー":
    train_size = 2000
elif vegetable == "そらまめ":
    train_size = 800
elif vegetable == "ミニトマト":
    train_size = 1500
else:
    train_size = 4000

In [4]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

# train_test["weekday"] = train_test.date.dt.weekday
# weekdays = pd.get_dummies(train_test["weekday"])
# train_test = train_test.drop(columns="weekday")
# train_test = pd.concat([train_test, weekdays], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

train = train_test[:pd.read_csv("./data/train.csv").shape[0]]

target_values = utils.get_target_values(train, vegetable)
changed_col = [1, 0] + [i for i in np.arange(2, target_values.shape[1])]
target_values = target_values[:, changed_col]

# Set Config, Params

In [7]:
rnn_options = {
    "target_values": target_values,
    "training_size": train_size,
}

rnn_config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "weight_decay": tune.choice([0, 1e-10, 1e-7, 1e-5, 1e-3]),
    "eps": tune.choice([1e-11, 1e-8, 1e-5, 1e-3, 1e-1]),
    "num_epochs": tune.choice([25, 50, 75, 100, 150]),
    "batch_size": tune.choice([16, train_size]),
    "wandb": {
        "project": f"{vegetable}",
        "api_key_file": "./wandb_api_key.txt"
    }
}

# Raytune

In [ ]:
utils.pipeline_raytune(rnn_options, rnn_config)